In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,recall_score,f1_score
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from sklearn.model_selection import KFold,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('mydata.csv')

In [3]:
y=df.Accident_Severity
df = df.drop(['Accident_Severity'], axis=1)

In [20]:
X = df.values

In [21]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [6]:
logistic_regression=LogisticRegression(class_weight="balanced",solver="newton-cg")

In [7]:
from sklearn.metrics import recall_score
from sklearn.metrics import  make_scorer
recall_scorer = make_scorer(recall_score,average="micro")

In [8]:
grid={"C":[0.00001,0.0001]}

logistic_regression_cv=GridSearchCV(logistic_regression,grid,cv=10,scoring=recall_scorer)
logistic_regression_cv.fit(X,y)

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='newton-cg', tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [1e-05, 0.0001]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn',
       scoring=make_scorer(recall_score, average=micro), verbose=0)

In [9]:
print("tuned hpyerparameters :(best parameters) ",logistic_regression_cv.best_params_)

tuned hpyerparameters :(best parameters)  {'C': 0.0001}


In [10]:
kf = KFold(n_splits=10)

In [14]:
def normalize(z):
    x = z.values #returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(np.vstack(x))
    return x_scaled

In [17]:
for i in range(0,9):
    df.iloc[:,i]=normalize(df.iloc[:,i])

In [22]:
# Grid search cross validation 

print("Train \t\t\t Test")
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    logistic_regression=LogisticRegression(class_weight="balanced",C=0.0001,solver="newton-cg")
    logistic_regression.fit(X_train,y_train)
    y_ = logistic_regression.predict(X_test)

    y_pred = logistic_regression.predict(X_train)
    
    print(accuracy_score(y_train,y_pred),"\t",accuracy_score(y_test,y_))


Train 			 Test
0.538779479699 	 0.569868995633
0.538162372387 	 0.599562363239
0.538648517258 	 0.577680525164
0.540350024307 	 0.536105032823
0.548128342246 	 0.514223194748
0.545940690326 	 0.505470459519
0.543266893534 	 0.507658643326
0.546669907632 	 0.496717724289
0.54132231405 	 0.562363238512
0.546183762761 	 0.540481400438


In [23]:
confusion_matrix(y_test,y_,labels=[1,2,3])

array([[  4,   1,   2],
       [ 22,   4,  45],
       [120,  20, 239]])

In [7]:
clf = svm.SVC(class_weight="balanced")

In [ ]:
grid={"C":[30,100],"gamma":[0,1]}

clf_cv=GridSearchCV(clf,grid,cv=5,scoring= recall_scorer)
clf_cv.fit(X,y)

In [ ]:
clf_cv.best_params_

Let's check out SVM

In [13]:

print("Train \t\t\t Test")
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf = svm.SVC(gamma=0.001,C=0.1,class_weight="balanced",kernel="linear")
    clf.fit(X_train, y_train) 
    y_ = clf.predict(X_test)

    y_pred = clf.predict(X_train)
    
    print(accuracy_score(y_train,y_pred),"\t",accuracy_score(y_test,y_))


Train 			 Test
0.53537563822 	 0.576419213974
0.520661157025 	 0.544857768053
0.537190082645 	 0.579868708972
0.545940690326 	 0.557986870897
0.546426835197 	 0.505470459519
0.553475935829 	 0.494529540481
0.556392805056 	 0.520787746171
0.551288283909 	 0.501094091904
0.537676227516 	 0.564551422319
0.540836169178 	 0.536105032823


In [10]:
confusion_matrix(y_test,y_,labels=[1,2,3])

array([[  4,   1,   2],
       [ 17,  10,  44],
       [ 90,  58, 231]])